# 처음 배우는 셸 스크립트#6

## 3부 예제와 함께 활용하는 셸 스크립트 활용
시스템 운영, 개발환경 구축, 클라우드 시스템 운영 시 발생하는 다양한 상황에서 셸 스크립트를 활용한다.  
상황 -> 방법 찾기 -> 스크립트 생성 -> 문제 해결  

## 8 시스템 구축
1. 사용자 계정을 만든다
2. 사용자 계정으로 시스템에 접속할 수 있도록 SSH Key를 생성한다
3. 여러 시스템의 시간대를 설정하고 동기화한다
4. 위 과정을 셸 스크립트로 작성한다

### 8.1 사용자 계정을 만들 때
##### 상황 
시스템 하나에 사용자 계정 하나만 만들면 상관 없지만, 여러 서버에 만들거나 여러 계정을 만든다면 시간이 많이 든다
##### 방법 찾기
셸 스크립트를  개발하기 위해서 프로세스 설계가 반드시 필요하다.  
필요한 정보  
- 사용자 계정 ID와 패스워드
- 사용자 계정 생성 명령어: useradd
- 패스워드 설정 명령어: passwd
프로세스
- 사용자 계정과 패스워드를 입력받는다
- 입력 정보가 없으면 에러 메시지를 보여주고 셸 스크립트를 종료한다
- 여러 명의 사용자 계정을 생성할 경우 for문을 사용한다
- 생성하고자 하는 사용자 계정이 이미 있는지 확인한다
- 사용자 계정이 없으면 사용자 계정을 생성하고 패스워드를 설정한다
- 사용자 계정이 있으면 계정이 있다고 메시지를 보여준다
##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash
## 하나의 시스템에 복수의 계정 생성
# 사용자 계정 및 패스워드가 입력되었는지 확인
if [[ -n $1 ]] && [[ -n $2 ]]
then

  UserList=($1)
  Password=($2)

  # for문을 이용하여 사용자 계정 생성
  for (( i=0; i < ${#UserList[@]}; i++ ))
  do
    # if문을 사용하여 사용자 계정이 있는지 확인
    if [[ $(cat /etc/passwd | grep ${UserList[$i]} | wc -l) == 0 ]]
    then
      # 사용자 생성 및 패스워드 설정
      useradd ${UserList[$i]}
      echo ${Password[$i]} | passwd ${UserList[$i]} --stdin
    else
      # 사용자가 있다고 메시지를 보여줌.
      echo "this user ${UserList[$i]} is existing."
    fi
  done

else
  # 사용자 계정과 패스워드를 입력하라는 메시지를 보여줌.
  echo -e 'Please input user id and password.\nUsage: adduser-script.sh "user01 user02" "pw01 pw02"'
fi


In [ ]:
%%bash
#!/bin/bash
## 여러 시스템에 하나의 동일한 계정 생성
## 이 스크립트를 사용하기 전, 서버의 도메인주소나 IP주소를 알아야 하며, ssh로 root계정에 접속할 수 있어야 한다.
for server in "host01 host02 host03"
do
  # 여러대의 시스템에 사용자 생성 및 패스워드 설정
  echo $server
  ssh root@$server "useradd $1"
  ssh root@$server "echo $2 | passwd $1 --stdin"
done


In [ ]:
%%bash
## 문제해결
bash adduser-script.sh
bash adduser-script.sh "user01 user02" "user01pw! user02pw!"
bash adduser-script.sh "user01 user02" "user01pw! user02pw!"

### 8.2 SSH Key를 여러 서버에 복사할 때
##### 상황 
서버들에 접속할 때 SSH 키를 주로 사용한다. 쉽고 빠르게 SSH 공개 키를 서버들에 복사하자
##### 방법 찾기
SSH키를 먼저 생성한 후 해당 키를 여러 서버에 복사한다  
이때 계정과 패스워드를 알고 있어야 한다.
필요한 정보  
- SSH Key 생성 명령어: ssh-keygen
- SSH Key 복사 명령어: ssh-copy-id
- 접속할 서버 정보: IP, 접속 계정 및 패스워드
프로세스
- 접속할 서버 정보 및 SSH 키, 공개 키 경로를 변수에 저장한다
- SSH Key를 생성한다
- 생성한 SSH 공개 키를 해당 서버에 복사한다
##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

# 접속할 서버 정보, SSH 키 경로, 공개키 경로를 변수에 저장
servers="host01 host02"
sshKey="$HOME/.ssh/key.pem"
sshPub="$HOME/.ssh/key.pem.pub"

# SSH Key 생성
ssh-keygen -q -N "" -f $sshKey

# 생성된 SSH Key를 해당 서버에 복사
for server in $servers
do
  echo $server
  sshpass -p "$1" ssh-copy-id -i $sshPub stack@$server
done

In [ ]:
## 문제 해결
bash send-new-ssh-key.sh stackpw
ssh stack@host01

### 8.3 다수의 서버에 NTP 서버를 설치할 때
##### 상황 
NTP는 Netowrk Time Protocol의 약자로 서버들의 시간을 동기화하기 위해 설치하는 서비스다.
이 서비스를 여러 서버에 설치해야 할 경우 셸 스크립트를 활용하자
##### 방법 찾기
NTP를 설치하기 위해서는 패키지 리포지토리에 접속할 수 있어야 한다.
필요한 정보  
- NTP 설치 명령어: 페도라 계열 리눅스 yum install ntp
- NTP 설치 명령어: 데비안 계열 리눅스 apt-get install ntp
- NTP 서버 정보, NTP를 설치할 대상 서버 정보
프로세스
- NTP를 설치할 대상 서버 정보를 변수에 저장한다
- 리눅스가 페도라 계열인지 데비안 계열인지 체크한다
- 페도라 계열이면 yum install로 ntp를 해당 서버에 설치한다
- 데비안 계열이면 apt-get install로 ntp를 해당 서버에 설치한다

##### 스크립트 생성
SSH를 통해 NTP를 설치하려면 root 권한이 있는 사용자 계정이나 root 계정을 사용해야 한다

In [ ]:
%%bash
#!/bin/bash

# NTP를 설치할 대상서버정보 저장
servers='host01 host02 host03'
cmd1='cat /etc/*release| grep ID_LIKE | sed "s/ID_LIKE=//;s/\"//g"' # ID_LIKE="fedora" 출력을 fedora로 바꿔줌
cmd2=''

for server in $servers; do
  # 해당 서버의 운영체제 타입 확인
  ostype=$(sshpass -p $1 ssh root@$server $cmd1)

  # 운영체제가 Fedora 계열인지 Debian 계열인지 체크
  if [[ $ostype == "fedora" ]]; then
    cmd2="yum install -y ntp"
  elif [[ $ostype == "debian" ]]; then
    cmd2="apt-get install -y ntp"
  fi
  
  # 해당 운영체제에 ntp 설치
  sshpass -p $1 ssh root@$server $cmd2
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
sh install-ntp.sh rootpw

### 8.4 다수의 서버에 Timezone을 설정할 때
##### 상황 
UTC를 기준으로 각 나라마다 설정하는 Timezone이 다르다.  
한국은 Asia/Seoul이다. 서버의 Timezoen이 설정되지 않은 경우, 설정해주자  
##### 방법 찾기
Timezone을 설정하기 위해 필요한 명령어와 프로세스를 알아본다
필요한 정보  
- Timezone을 확인하는 명령어: datetimectl status
- Timezone을 설정하는 명령어: datetimectl set-timezone
프로세스
- 파라미터로 입력받은 Timezone을 변수에 저장한다
- 해당 서버의 Timezone을 확인하기 위한 명령어를 저장한다
- For문을 돌면서 해당 서버의 Timezone을 확인한다
- 입력받은 Timezone과 해당 서버의 Timezone이 일치하는지 확인한다
- 일치하지 않으면 입력받은 Timezone으로 해당 서버의 Timezone을 변경한다

##### 스크립트 생성

In [ ]:
%%bash
#!/bin/bash

# Timezone을 설정할 대상정보 및 명령어 저장
servers="host01 host02 host03"
cmd1="timedatectl status | grep 'Time zone'"
cmd2="timedatectl set-timezone $1"

# timezone 또는 패스워드 둘 중 하나라도 입력하지 않았다면 스크립트 종료
if [[ -z $1 ]] || [[ -z $1 ]]; then
  echo -e 'Please input timezone and password\nUsage: sh set-timezone.sh Seoul/Asia password'
  exit; 
fi

for server in $servers
do
  # 해당 서버의 설정된 timezone 정보 조회
  timezone=$(sshpass -p $2 ssh root@$server "$cmd1" | awk '{print $3}')
  echo "$server: $timezone"

  # 설정하고자 하는 timezone과 조회된 timezone이 다른지 확인
  if [[ $timezone != $1 ]]
  then
    # timezone이 서로 다르면 해당 서버에 입력받은 timezone으로 설정 
    sshpass -p $2 ssh root@$server $cmd2
    echo "$server timezone changed to $1"
  fi
done



In [ ]:
%%bash
#!/bin/bash
## 문제 해결
timedatectl list-timezones
bash set-timezone.sh
bash set-timezone.sh Asia/Seoul rootpw

### 8.5 CLI를 이용하여 오픈스택 인스턴스를 생성할 때 : 생략
### 8.6 패키지 리포지터리 다운로드할 때
##### 상황 
회사에 구축되는 리눅스 서버는 인터넷이 되지 않는 환경에서 구축되는 경우가 많다.  
회사 내부에 패키지 리포지터리를 구성하는 경우가 많다. 그런 경우 시스템 엔지니어는 인터넷이 되는 DMZ 구간의 서버에서 외부 리포지터리와 내부 리포지터리를 동기화해야 한다.  
이런 경우, 셸 스크립트를 이용하면 쉽게 리포지터리의 패키지들을 다운로드할 수 있다. 데비안 계열도 방법이 있지만, 많이 사용하지 않으므로 레드햇 리눅스에 대한 동기화만 알아본다.
##### 방법 찾기
패키지 리포지터리를 다운받는 명령어를 알아본다
필요한 정보  
- 패키지 리포지터리 동기화 명령어: reposync
- 디렉터리의 리포지터리화하는 명령어: createrepo
프로세스
- 동기화할 리포지터리는 외부로부터 입력받아 변수에 저장한다
- 리포지터리를 저장할 경로를 저장한다
- 운영체제 버전을 확인한다
- 리포지터리를 동기화한다
- 동기화가 끝나면 리포지터리를 다운받은 경로를 createrepo를 통해 리포지터리화 한다

##### 스크립트 생성
리포지터리 서버는 주로 아파치 웹 서버를 사용한다. 레드햇 리눅스와 같은 경우, 서브스크립션을 등록해야 한다.

In [ ]:
%%bash
#!/bin/bash

# 레파지토지 목록을 입력받지 않고, 파일에 직접 입력해도 됨
repolist=$1
repopath=/var/www/html/repo/
osversion=$(cat /etc/redhat-release | awk '{print $(NF-1)}')

# 레파지토리 입력이 없으면 메시지를 보여주고 스크립트 종료
if [[ -z $1 ]]; then
  echo "Please input repository list. You can get repository from [yum repolist]"
  echo "Rhel7 Usage: reposync.sh \"rhel-7-server-rpms\""
  echo "Rhel8 Usage: reposync.sh \"rhel-8-for-x86_64-baseos-rpms\""
  exit;
fi

# 운영체제 버전에 따라 입력한 레포지토리만큼 동기화를 함.
for repo in $repolist; do
  # OS가 Rhel7일 경우
  if [ ${osversion:0:1} == 7 ]; then
    reposync --gpgcheck -l -n --repoid=$repo --download_path=$repopath
  # OS가 Rhel8일 경우
  elif [ ${osversion:0:1} == 8 ]; then
    reposync --download-metadata --repo=$repo -p $repopath
  fi
  # 해당 디렉토리를 레파지토리화한다.
  createrepo $repopath$repo
done


In [ ]:
%%bash
#!/bin/bash
## 문제 해결
sudo bash reposync.sh
sudo bash reposync.sh "rhel-7-server-rpms"